# Advent of Code 2023

Imports and helper function

In [ ]:
from aoc import *
import re
from aocd.models import Puzzle as AOCDPuzzle


def Puzzle(day: int, year: int = 2023):
    return AOCDPuzzle(year=year, day=day)

# Day 1 - Trebuchet?!

In [ ]:
p = Puzzle(day=1)

In [ ]:
WORDS = {
    "one": 1,
    "two": 2,
    "three": 3,
    "four": 4,
    "five": 5,
    "six": 6,
    "seven": 7,
    "eight": 8,
    "nine": 9,
}


def words_to_digits(s: str):
    while True:
        first = 10**9
        word = None
        digit = None
        for w, d in WORDS.items():
            idx = s.find(w)
            if idx >= 0 and idx < first:
                first, word, digit = idx, w, d
        if word is None:
            break
        s = s.replace(word, str(digit), 1)
    return s


def trebuchet(x):
    def digits(line):
        return [int(d) for d in line if d.isdigit()]

    lines = [digits(line) for line in x.splitlines()]
    return sum(digits[0] * 10 + digits[-1] for digits in lines)


assert trebuchet(p.examples[0].input_data) == 142
assert (
    trebuchet(
        words_to_digits(
            """two1nine
eightwothree
abcone2threexyz
xtwone3four
4nineeightseven2
zoneight234
7pqrstsixteen"""
        )
    )
    == 281
)

In [ ]:
print(trebuchet(p.input_data), trebuchet(words_to_digits(p.input_data)))

# Day 2 - Cube Conundrum

In [ ]:
p = Puzzle(day=2)

In [ ]:
def parse_cube_game(line):
    def parse_round(r):
        return dict(
            (tpl[1], int(tpl[0]))
            for tpl in [tuple(x for x in t.split(" ")) for t in r.split(", ")]
        )

    game, rest = line.strip().split(": ")
    game = int(game[5:])
    rounds = [parse_round(r) for r in rest.split("; ")]
    return game, rounds


def valid_round(round: dict, cubes: dict):
    return all(balls <= cubes[color] for color, balls in round.items())


def cube_game(data, cubes: dict = {"red": 12, "green": 13, "blue": 14}):
    valid_games = 0
    total_power = 0
    for line in data.splitlines():
        game, rounds = parse_cube_game(line)
        if all(valid_round(r, cubes) for r in rounds):
            valid_games += game
        min_balls = defaultdict(int)
        for r in rounds:
            for color, count in r.items():
                min_balls[color] = max(min_balls[color], count)
        power = min_balls["red"] * min_balls["green"] * min_balls["blue"]
        total_power += power
    return valid_games, total_power


assert (8, 2286) == cube_game(p.examples[0].input_data)

In [ ]:
p.answers = cube_game(p.input_data)

# Day 3 - Gear Ratios

In [ ]:
p = Puzzle(day=3)

In [ ]:
def parse_gears(input):
    total = 0
    gear_ratio = 0
    schematic = [l.strip() for l in input.splitlines()]
    gears = defaultdict(set)

    def is_part(row, start, end):
        result = False
        for x in range(start, end):
            for xx, yy in neighbors8((x, row)):
                if xx < 0 or xx >= len(line) or yy < 0 or yy >= len(schematic):
                    continue
                n = schematic[yy][xx]
                if n.isdigit() or n == ".":
                    continue
                if n == "*":
                    gears[(xx, yy)].add((row, start, int(schematic[row][start:end])))
                result = True
        return result

    for row, line in enumerate(schematic):
        for m in re.finditer(r"\d+", line):
            start, end = m.span()
            num = int(line[start:end])
            if is_part(row, start, end):
                total += num
    for pos, parts in gears.items():
        if len(parts) != 2:
            continue
        parts = list(parts)
        gear_ratio += parts[0][-1] * parts[1][-1]
    return total, gear_ratio


assert (4361, 467835) == parse_gears(p.examples[0].input_data)

In [ ]:
p.answers = parse_gears(p.input_data)

# Day 4 - Scratchcards

In [ ]:
p = Puzzle(day=4)

In [ ]:
def scratchcards(data):
    total = 0
    copies = defaultdict(lambda: 1)
    for line in data.splitlines():
        m = re.match(r"^Card\s+(\d+): ([\d\s]+) \| ([\d\s]+)", line.strip())
        assert m, line
        card, winners, have = (
            int(m.group(1)),
            set(vector(m.group(2))),
            set(vector(m.group(3))),
        )
        hits = len(winners & have)
        points = 2 ** (hits - 1) if hits > 0 else 0
        total += points
        copies[card]
        for x in range(card + 1, card + hits + 1):
            copies[x] += copies[card]
    return total, sum(copies.values())


assert (13, 30) == scratchcards(p.examples[0].input_data)

In [ ]:
p.answers = scratchcards(p.input_data)

# Day 5 - If You Give A Seed A Fertilizer

In [ ]:
p = Puzzle(day=5)

In [ ]:
def parse_seeds(data):
    lines = data.splitlines()
    seeds = vector(lines[0][6:])
    maps = []
    m = None
    for line in lines[2:]:
        if not line.strip():
            continue
        if line.endswith("map:"):
            maps.append([])
            m = maps[-1]
            continue
        m.append(vector(line))
    return seeds, maps


def map_val(val, m):
    for dest, src, length in m:
        offset = val - src
        if 0 <= offset < length:
            return dest + offset
    return val


def map_range(start: int, count: int, m: tuple[int]):
    queue = [(start, count)]
    while queue:
        (start, count) = queue.pop()
        for dest, src, length in m:

            def translate(s):
                offset = s - src
                assert offset >= 0, f"translate: {s} on {(dest, src, length)}"
                return dest + offset

            x = max(src, start)
            y = min(src + length, start + count)
            if not x < y:
                continue
            yield translate(x), y - x
            if x > start:
                queue.append((start, x - start))
            if y < start + count:
                queue.append((y, start + count - y))
            break
        else:
            yield (start, count)


def start_garden(seeds, maps):
    best = BIG
    for dest in seeds:
        for m in maps:
            dest = map_val(dest, m)
        best = min(best, dest)
    part2 = BIG
    for start, count in grouper(seeds, 2):
        if count > 1_000_000:
            continue
        for dest in range(start, start + count):
            for m in maps:
                dest = map_val(dest, m)
            part2 = min(part2, dest)
    return best, part2


assert (35, 46) == start_garden(*parse_seeds(p.examples[0].input_data))

In [ ]:
p.answer_a = start_garden(*parse_seeds(p.input_data))[0]

In [ ]:
seeds, maps = parse_seeds(p.input_data)
best = 10**10
for node, count in grouper(seeds, 2):
    dests = [(node, count)]
    for i, m in enumerate(maps, 1):
        dests = [x for d in dests for x in map_range(*d, m)]
    best = min(best, min(dests)[0])

p.answer_b = best

# Day 6 - Wait For It

In [ ]:
p = Puzzle(day=6)

In [ ]:
def boat_race(data):
    d = array(data)
    races = dict(zip(d[0][1:], d[1][1:]))

    def ways_to_win(time, distance):
        wins = 0
        winning = False
        for speed in range(time + 1):
            if speed * (time - speed) > distance:
                wins += 1
                winning = True
                continue
            if winning:
                break
        return wins

    results = []
    for time, distance in races.items():
        results.append(ways_to_win(time, distance))
    part_a = multiply(results)
    d = array(data.replace(" ", ""))
    races = dict(zip(d[0][1:], d[1][1:]))
    results = []
    for time, distance in races.items():
        results.append(ways_to_win(time, distance))
    return part_a, multiply(results)


assert (288, 71503) == boat_race(p.examples[0].input_data)

In [ ]:
p.answers = boat_race(p.input_data)

# Day 7 - Camel Cards

In [ ]:
p = Puzzle(day=7)

In [ ]:
def card_rank(card: str, jokers: bool = False):
    if not jokers:
        return "AKQJT98765432".index(card)
    return "AKQT98765432J".index(card)


def hand_order(hand: str, jokers: bool = False):
    return tuple(card_rank(c, jokers) for c in hand)


HAND_RANK = [(5,), (4, 1), (3, 2), (3, 1), (2, 2), (2, 1), (1, 1)]


def hand_type(hand: str, use_jokers: bool = False):
    c = Counter(hand)
    if use_jokers:
        # Eliminate Jokers, add them to card with highest count
        num_jokers = c.get("J", 0)
        if num_jokers < 5:  # We have at least one other card
            c.subtract({"J": num_jokers})
            card = c.most_common(1)[0][0]
            c.update({card: num_jokers})
    counts = tuple(count for card, count in c.most_common(2) if count > 0)
    return HAND_RANK.index(counts)


def camel_cards(data):
    hands = array(data)
    ordered = sorted(hands, key=lambda t: hand_order(str(t[0])))
    ranked = sorted(ordered, key=lambda t: hand_type(str(t[0])))
    part_a = sum([i * t[1] for i, t in enumerate(reversed(ranked), 1)])

    ordered = sorted(hands, key=lambda t: hand_order(str(t[0]), True))
    ranked = sorted(ordered, key=lambda t: hand_type(str(t[0]), True))
    part_b = sum([i * t[1] for i, t in enumerate(reversed(ranked), 1)])
    return part_a, part_b


assert 6440, 5905 == camel_cards(p.examples[0].input_data)

In [ ]:
p.answers = camel_cards(p.input_data)

# Day 8 - Haunted Wasteland

In [ ]:
p = Puzzle(day=8)

In [ ]:
def wasteland(data):
    path, _, *rest = data.splitlines()
    graph = dict()
    for line in rest:
        m = re.match(r"(\w+) = \((\w+), (\w+)\)", line)
        assert m
        node, left, right = m.groups()
        graph[node] = (left, right)
    assert "AAA" in graph and "ZZZ" in graph
    node, num_steps = "AAA", 0
    while node != "ZZZ":
        index = 0 if path[num_steps % len(path)] == "L" else 1
        node = graph[node][index]
        num_steps += 1
    answer_a = num_steps
    starts = [k for k in graph.keys() if k.endswith("A")]
    answer_b = []
    # Solve each path's "Z" problem and calculate the LCM of all
    for i, node in enumerate(starts):
        num_steps = 0
        while not node.endswith("Z"):
            index = 0 if path[num_steps % len(path)] == "L" else 1
            node = graph[node][index]
            num_steps += 1
        answer_b.append(num_steps)
    # warns: "coerced int64 value 18625484023687 for 2023/08"
    return answer_a, np.lcm.reduce(answer_b, dtype="int64")


assert 6, 6 == wasteland(
    """LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ)"""
)

In [ ]:
p.answers = wasteland(p.input_data)

# Day 9 - Mirage Maintenance

In [ ]:
p = Puzzle(day=9)

In [ ]:
def oasis(data):
    data = array(data)

    def solve(row):
        diffs = tuple(row[i] - row[i - 1] for i in range(1, len(row)))
        if all(d == 0 for d in diffs):
            return (row[0], row[-1])
        ans = solve(diffs)
        return (row[0] - ans[0], row[-1] + ans[-1])

    answers = [solve(row) for row in data]
    return sum(a[1] for a in answers), sum(a[0] for a in answers)


assert (114, 2) == oasis(p.examples[0].input_data)

In [ ]:
p.answers = oasis(p.input_data)

# Day 10 - Pipe Maze

Here we are introduced to Pick's theorem and the Shoelace algorithm, also useful later in the month.

In [ ]:
p = Puzzle(day=10)

In [ ]:
# UP, DOWN, LEFT, RIGHT valid neighbors for each segment
PIPE_LINKS = {
    "|": ("|7F", "|LJ", "", ""),
    "-": ("", "", "-LF", "-J7"),
    "L": ("|7F", "", "", "-J7"),
    "J": ("|7F", "", "-LF", ""),
    "7": ("", "|LJ", "-LF", ""),
    "F": ("", "|LJ", "", "-J7"),
    "S": ("|7F", "|LJ", "-LF", "-J7"),
}


def shoelace(points):
    return (
        sum(X(p0) * Y(p1) - Y(p0) * X(p1) for p0, p1 in pairwise(points + [points[0]]))
        // 2
    )
    # Original, before turning into a list comprehension:
    area = 0
    xs = [X(p) for p in points] + [X(points[0])]
    ys = [Y(p) for p in points] + [Y(points[0])]
    for i in range(len(points)):
        area += xs[i] * ys[i + 1] - ys[i] * xs[i + 1]
    return abs(area) // 2


def pipe_maze(data):
    lines = data.splitlines()
    maze = dict(
        ((x, y), pipe) for y, line in enumerate(lines) for x, pipe in enumerate(line)
    )
    start = next((x, y) for (x, y), pipe in maze.items() if pipe == "S")

    def pipe_neighbors(p):
        segment = maze[p]
        for index, (dx, dy) in enumerate([(0, -1), (0, 1), (-1, 0), (1, 0)]):
            q = (X(p) + dx, Y(p) + dy)
            c = maze.get(q)
            if c is None or c not in PIPE_LINKS[segment][index]:
                continue
            yield q, c

    path = set(start)
    vertices = [(start)]
    pos = start
    b = 1
    while True:
        l = len(path)
        for p, c in pipe_neighbors(pos):
            if p in path:
                continue
            path.add(p)
            if c in "LJF7":
                vertices.append(p)
            pos = p
            b += 1
            break
        else:
            break
    return len(path) // 2, shoelace(vertices) + 1 - b // 2


assert (80, 10) == pipe_maze(
    """FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJIF7FJ-
L---JF-JLJIIIIFJLJJ7
|F|F-JF---7IIIL7L|7|
|FFJF7L7F-JF7IIL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L"""
)

In [ ]:
p.answers = pipe_maze(p.input_data)

# Day 11 - Cosmic Expansion

In [ ]:
p = Puzzle(day=11)

In [ ]:
def galaxies(data, empty_mult=2):
    data = data.splitlines()
    rows, cols = len(data), len(data[0])
    empty_cols = set(i for i in range(cols) if not any(r[i] == "#" for r in data))
    empty_rows = set(i for i, r in enumerate(data) if "#" not in r)
    galaxies = set(
        (c, r)
        for r, line in enumerate(data)
        for c, glyph in enumerate(line)
        if glyph == "#"
    )
    total = 0
    for g0, g1 in combinations(galaxies, 2):
        x0, x1 = min(X(g0), X(g1)), max(X(g0), X(g1))
        y0, y1 = min(Y(g0), Y(g1)), max(Y(g0), Y(g1))
        distance = sum(
            empty_mult if c in empty_cols else 1 for c in range(x0, x1)
        ) + sum(empty_mult if r in empty_rows else 1 for r in range(y0, y1))
        total += distance
    return total


assert 374 == galaxies(p.examples[0].input_data, 2)
assert 1030 == galaxies(p.examples[0].input_data, 10)
assert 8410 == galaxies(p.examples[0].input_data, 100)

In [ ]:
p.answers = galaxies(p.input_data, 2), galaxies(p.input_data, 1_000_000)

# Day 12 - Hot Springs

Needed help to solve the second part.  First part solution uses `int.bit_count` so needs Python 3.10

In [ ]:
p = Puzzle(day=12)

In [ ]:
# Count / group set bits; too slow for Part 2
def one_spring(line: str, groups=list[int], debug=False):
    line = line.strip(".").replace(".", "0").replace("#", "1")
    indexes = [i for i, c in enumerate(line) if c == "?"]
    test = list(line)
    answers = 0
    fixed_ones = sum(1 for c in test if c == "1")
    need_ones = sum(groups)
    for val in range(2 ** len(indexes)):
        if int.bit_count(val) + fixed_ones != need_ones:
            continue
        for bit, offset in enumerate(indexes):
            test[offset] = "1" if (val & 1 << bit) else "0"
        g = []
        prev = None
        for c in test:
            if c == "1":
                if c != prev:
                    g.append(0)
                g[-1] += 1
            prev = c
        if g == groups:
            answers += 1
            if debug:
                print(val, "".join(test), groups)
    return answers


# Recursive solver with memoization, inspired by two other solutions seen in the wild.
def recursive_spring(line: str, groups: list[int], seen: dict):
    key = (line, tuple(groups))
    if (cached := seen.get(key)) is not None:
        return cached
    if not groups:
        return "#" not in line
    if sum(groups) + len(groups) - 1 > len(line):
        return 0
    if line[0] == "#":
        g0 = groups[0]
        if (
            len(line) >= g0
            and "." not in line[:g0]
            and (len(line) == g0 or line[g0] != "#")
        ):
            return recursive_spring(line[g0 + 1 :], groups[1:], seen)
        return 0
    if line[0] == ".":
        return recursive_spring(line[1:], groups, seen)
    count = seen[(line, tuple(groups))] = recursive_spring(
        line[1:], groups, seen
    ) + recursive_spring("#" + line[1:], groups, seen)
    return count


def springs(data, mult=1, debug=False):
    answer = 0
    seen = dict()
    for spring, *groups in array(data):
        arrangements = recursive_spring("?".join([spring] * mult), groups * mult, seen)
        answer += arrangements
        if debug:
            print(mult, spring, groups, arrangements)
    return answer


assert 21 == springs(
    """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1"""
)

assert 525152 == springs(
    """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1""",
    5,
)

In [ ]:
p.answers = springs(p.input_data), springs(p.input_data, 5)

# Day 13 - Point of Incidence

Using bitops again

In [ ]:
p = Puzzle(day=13)

In [ ]:
def smudge_symmetry(a: list[int], bits=1):
    for x in range(len(a) - 1):
        width = min(x, len(a) - x - 2)
        if width < 0:
            continue
        smudges = sum(int.bit_count(a[x - i] ^ a[x + i + 1]) for i in range(width + 1))
        if smudges == bits:
            return x + 1
    return None


def mirrors(data, bits=0):
    patterns = data.split("\n\n")
    total = 0
    for pattern in patterns:
        pattern = pattern.replace("#", "1").replace(".", "0")
        rows = pattern.splitlines()
        cols = [int("".join(row[i] for row in rows), 2) for i in range(len(rows[0]))]
        rows = [int(row, 2) for row in rows]
        answer = None
        if (row := smudge_symmetry(rows, bits)) is not None:
            answer = 100 * row
        else:
            answer = smudge_symmetry(cols, bits)
        assert answer is not None, f"No symmetry: {pattern}"
        total += answer
    return total


assert 405 == mirrors(p.examples[0].input_data)
assert 400 == mirrors(p.examples[0].input_data, 1)

In [ ]:
p.answers = mirrors(p.input_data), mirrors(p.input_data, 1)

# Day 14 - Parabolic Reflector Dish

In [ ]:
p = Puzzle(day=14)

In [ ]:
def rocks(input):
    seen = dict()
    steps = []

    def tilt(input, direction):
        dx, dy = direction
        rows = input.splitlines()
        oodles = input.count("O")
        if dx == 0:
            result = [[row[x] for row in rows] for x in range(len(rows[0]))]
        else:
            result = [[c for c in line] for line in input.splitlines()]
        tilt = dx or dy
        for y, row in enumerate(result):
            edge = 0 if tilt < 0 else len(row) - 1
            rocks = sorted([x for x, c in enumerate(row) if c == "O"], reverse=tilt > 0)
            oldrow = row[:]
            for x in rocks:
                new_x = first(
                    [
                        idx - tilt
                        for idx in range(x + tilt, edge + tilt, tilt)
                        if row[idx] != "."
                    ],
                    edge,
                )
                if new_x != x:
                    row[new_x], row[x] = "O", "."
                    edge = new_x
            assert row.count("O") == len(
                rocks
            ), f"{direction} {y}/{len(result)}: {len(rocks)}:\n{''.join(oldrow)}\n{''.join(row)}"
        if dx:
            result = "\n".join(["".join(row) for row in result])
        else:
            result = "\n".join(
                ["".join(col[y] for col in result) for y in range(len(result))]
            )
        assert result.count("O") == oodles
        return result

    result = tilt(input, (0, -1))

    def calc_load(input):
        data = input.splitlines()
        rows = len(data)
        return sum(row.count("O") * (rows - y) for y, row in enumerate(data))

    part_a = calc_load(result)
    CYCLES = 10**9

    output = input
    for _ in range(CYCLES):
        cycle = _ + 1
        orig = output
        for direction in ((0, -1), (-1, 0), (0, 1), (1, 0)):
            output = tilt(output, direction)
        if output in seen:
            break
        seen[output] = cycle
        steps.append(output)

    cycle_start = seen[output]
    cycle_len = cycle - cycle_start
    residual = (CYCLES - cycle_start) % cycle_len
    return part_a, calc_load(steps[cycle_start + residual - 1])


assert 136, 64 == rocks(p.examples[0].input_data)

In [ ]:
p.answers = rocks(p.input_data)

# Day 15 - Lens Library

In [ ]:
p = Puzzle(day=15)

In [ ]:
def hash_algo(data):
    value = 0
    for c in data:
        value += ord(c)
        value = (value * 17) % 256
    return value


def hash_sequence(data):
    return sum(hash_algo(step) for step in data.split(","))


def hash_map(data):
    boxes = [None] * 256
    for step in data.split(","):
        if step.endswith("-"):
            label = step[:-1]
            op = "-"
        else:
            label, f = step.split("=")
            f = int(f)
            op = "="
        idx = hash_algo(label)
        box = boxes[idx]
        if box is None:
            box = boxes[idx] = dict()
        if op == "-":
            box.pop(label, None)
        else:
            box[label] = f
    return sum(
        i * j * f
        for i, box in enumerate(boxes, 1)
        for j, f in enumerate(box.values() if box is not None else [], 1)
    )


assert 52 == hash_algo("HASH")
assert 1320 == hash_sequence("rn=1,cm-,qp=3,cm=2,qp-,pc=4,ot=9,ab=5,pc-,pc=6,ot=7")
assert 145 == hash_map("rn=1,cm-,qp=3,cm=2,qp-,pc=4,ot=9,ab=5,pc-,pc=6,ot=7")

In [ ]:
p.answer_a = hash_sequence(p.input_data)
p.answer_b = hash_map(p.input_data)

# Day 16 - The Floor Will Be Lava

In [ ]:
p = Puzzle(day=16)

In [ ]:
UP, DOWN, LEFT, RIGHT = (0, -1), (0, 1), (-1, 0), (1, 0)
REFLECT = {
    "\\": {UP: LEFT, DOWN: RIGHT, LEFT: UP, RIGHT: DOWN},
    "/": {UP: RIGHT, DOWN: LEFT, LEFT: DOWN, RIGHT: UP},
}


def beams(input, start=(0, 0), dir=RIGHT):
    tiles = input.splitlines()

    def legal(x, y):
        return x >= 0 and x < len(tiles[0]) and y >= 0 and y < len(tiles)

    beams = [(start, dir)]
    seen = defaultdict(set)
    while beams:
        (x, y), (dx, dy) = beams.pop()
        assert legal(x, y), f"pos={(x, y)} dir={(dx, dy)} beams={beams} seen={seen}"
        if (dx, dy) in seen[(x, y)]:
            continue
        seen[(x, y)].add((dx, dy))
        tile = tiles[y][x]
        moves = []
        if tile == "." or (tile == "-" and dy == 0) or (tile == "|" and dx == 0):
            moves.append(((x + dx, y + dy), (dx, dy)))
        elif tile == "-":
            moves.append(((x - 1, y), LEFT))
            moves.append(((x + 1, y), RIGHT))
        elif tile == "|":
            moves.append(((x, y - 1), UP))
            moves.append(((x, y + 1), DOWN))
        else:
            assert tile in REFLECT
            dx, dy = REFLECT[tile][(dx, dy)]
            moves.append(((x + dx, y + dy), (dx, dy)))
        for (x, y), (dx, dy) in moves:
            if not legal(x, y):
                continue
            beams.append(((x, y), (dx, dy)))
    return len(seen)


def best_beam(input):
    tiles = input.splitlines()
    width = len(tiles[0])
    height = len(tiles)
    starts = []
    for x in range(width):
        starts.append(((x, 0), DOWN))
        starts.append(((x, height - 1), UP))
    for y in range(height):
        starts.append(((0, y), RIGHT))
        starts.append(((width - 1, y), LEFT))
    return max(beams(input, start, pos) for start, pos in starts)


assert 46 == beams(p.examples[0].input_data)
assert 51 == best_beam(p.examples[0].input_data)

In [ ]:
p.answers = beams(p.input_data), best_beam(p.input_data)

# Day 17 - Clumsy Crucible

I definitely used some help on part 2 to figure out the "state" involved position + direction + step count.  The solution is still rather slow, taking ~12s to run so there must be some possible further optimizations.

In [ ]:
p = Puzzle(day=17)

In [ ]:
from dataclasses import dataclass
from typing import Optional

UP, DOWN, LEFT, RIGHT = (0, -1), (0, 1), (-1, 0), (1, 0)
DIRS = [RIGHT, DOWN, LEFT, UP]


@dataclass(order=True, frozen=True)
class State:
    pos: tuple[int, int] = (0, 0)
    idx: int = 0
    steps: int = 0

    def __str__(self):
        return f"State({self.pos}, {'RDLU'[self.idx]}, {self.steps})"

    def __repr__(self):
        return f"State({self.pos}, {'RDLU'[self.idx]}, {self.steps})"


def lava(input, minsteps=0, maxsteps=3):
    data = input.splitlines()
    sparse = {(x, y): int(c) for y, line in enumerate(data) for x, c in enumerate(line)}
    width, height = len(data[0]), len(data)

    def move(pos, dir):
        x, y = pos
        dx, dy = dir
        return x + dx, y + dy

    def legal(pos):
        x, y = pos
        return 0 <= x < width and 0 <= y < height

    def moves(s: State):
        if 0 < s.steps < maxsteps:
            npos = move(s.pos, DIRS[s.idx])
            if legal(npos):
                yield State(pos=npos, idx=s.idx, steps=s.steps + 1)
        if s.steps == 0 or s.steps >= minsteps:
            for didx in (-1, 1) if s.steps > 0 else (0, 1, 2, 3):
                nidx = (s.idx + didx) % 4
                npos = move(s.pos, DIRS[nidx])
                if legal(npos):
                    yield State(pos=npos, idx=nidx, steps=1)

    def h_func(s: State):
        return s.pos != (width - 1, height - 1) or s.steps < minsteps

    def cost(_, s: State):
        return sparse.get(s.pos, 10**9)

    path = Astar(State(), moves, h_func, cost)
    return sum(sparse[s.pos] for s in path[1:])


assert 102 == lava(p.examples[0].input_data)
assert 94 == lava(p.examples[0].input_data, 4, 10)

In [ ]:
p.answers = lava(p.input_data, 0, 3), lava(p.input_data, 4, 10)

# Day 18 - Lavaduct Lagoon

In [ ]:
p = Puzzle(day=18)

In [ ]:
DIRECTIONS = {"U": (0, -1), "D": (0, 1), "L": (-1, 0), "R": (1, 0)}


def lagoon(input: str):
    def area(instructions):
        x, y = (0, 0)
        vertexes = [(0, 0)]
        perimiter = 0
        for udlr, distance in instructions:
            dx, dy = DIRECTIONS[udlr]
            x, y = x + distance * dx, y + distance * dy
            vertexes.append((x, y))
            perimiter += distance
        return shoelace(vertexes) + perimiter // 2 + 1

    data = array(input)
    part1 = area([(t[0], t[1]) for t in data])

    def decode_hex(s):
        s = s.strip("#()")
        distance = int(s[:5], 16)
        direction = "RDLU"[int(s[-1])]
        return direction, distance

    part2 = area([decode_hex(t[2]) for t in data])
    return part1, part2


assert (62, 952408144115) == lagoon(p.examples[0].input_data)

In [ ]:
p.answers = lagoon(p.input_data)

# Day 19 - Aplenty

Took me a while to figure out part 2 of this one

In [ ]:
p = Puzzle(day=19)

In [ ]:
import operator


def score_workflow(flow: dict):
    return multiply(x.bit_count() for x in flow.values())


def make_bits(i):
    return (1 << i) - 1


def aplenty(data):
    p1, p2 = data.split("\n\n")
    rules = {}
    raw_rules = {}

    def make_rule(token: str):
        m = re.match("(\w+)([<>])([\d]+):(\w+)", token)
        if m:
            label, op, val, dest = m.groups()
            op = {"<": operator.lt, ">": operator.gt}[op]
            val = int(val)
            return lambda x: dest if op(x.get(label), val) else None
        return lambda x: token

    for rule in p1.splitlines():
        m = re.match(r"(\w+)\{([^}]+)\}", rule)
        assert m, rule
        name, tokens = m.groups()
        tokens = tokens.split(",")
        rules[name] = [make_rule(t) for t in tokens]
        raw_rules[name] = tokens

    def check_expr(obj):
        label = "in"
        while label not in "AR":
            for r in rules[label]:
                dest = r(obj)
                if dest is not None:
                    label = dest
                    break
            else:
                result = "A"
        return label == "A"

    parts = []
    for line in p2.splitlines():
        part = {}
        for token in line.strip("{}").split(","):
            k, v = token.split("=")
            part[k] = int(v)
        parts.append(part)

    total = 0
    for p in parts:
        if check_expr(p):
            total += sum(p.values())

    part2 = 0
    queue = [("in", dict((k, make_bits(4000)) for k in "xmas"))]
    while queue:
        label, workflow = queue.pop()
        if label == "R":
            continue
        if label == "A":
            part2 += score_workflow(workflow)
            continue
        for token in raw_rules[label]:
            m = re.match(r"(\w+)([<>])([\d]+):(\w+)", token)
            if m:
                xmas, op, val, dest = m.groups()
                val = int(val)
                bits = workflow[xmas]
                mask = make_bits(val)
                if op == "<":
                    mask = make_bits(val - 1)
                elif op == ">":
                    mask = ~make_bits(val)
                if (nbits := bits & mask) != 0:
                    # Matching workflow
                    c = workflow.copy()
                    c.update({xmas: nbits})
                    queue.append((dest, c))
                # No match; next rule gets residual
                workflow.update({xmas: bits & ~mask})
            else:
                queue.append((token, workflow))

    assert part2 < 4000**4, part2
    return total, part2


assert 19114, 167409079868000 == aplenty(p.examples[0].input_data)

In [ ]:
p.answers = aplenty(p.input_data)

# Day 20 - Pulse Propagation

In [ ]:
p = Puzzle(day=20)

In [ ]:
def pulse(data):
    outputs = defaultdict(set)
    outputs["button"].add("broadcaster")
    inputs = defaultdict(set)
    inputs["broadcaster"].add("button")
    types = {}
    for line in data.splitlines():
        name, dests = line.split(" -> ", 1)
        dests = dests.split(", ")
        if name[0] in "%&":
            type, name = name[0], name[1:]
        else:
            type = None
        types[name] = type
        outputs[name].update(dests)
        for dest in dests:
            inputs[dest].add(name)

    def make_states(types, inputs):
        states = {}
        for name, type in types.items():
            if type == "%":
                states[name] = False
            elif type == "&":
                states[name] = dict((i, False) for i in inputs[name])
        return states

    part1 = [0, 0]
    counter = Counter()
    seen = dict()
    upstream = set(src for src, dest in outputs.items() if "rm" in dest)
    states = make_states(types, inputs)

    for press in range(1, 10**9):
        work = deque([("button", "broadcaster", False)])
        counter.clear()
        while work:
            sender, module, pulse = work.popleft()
            if press <= 1000:
                part1[pulse] += 1
            if pulse:
                counter.update((sender,))
            type = types.get(module)
            if type is None:
                for name in outputs[module]:
                    work.append((module, name, pulse))
            elif type == "%":
                if not pulse:
                    states[module] = not states[module]
                    for dest in outputs[module]:
                        work.append((module, dest, states[module]))
            elif type == "&":
                state = states[module]
                state[sender] = pulse
                nhigh = sum(state.values())
                for dest in outputs[module]:
                    work.append((module, dest, not all(state.values())))

        if upstream:
            matches = [k for k in upstream if counter.get(k) == 1]
            for k in matches:
                seen[k] = press
                upstream.remove(k)
            if not upstream:
                break
        elif press == 1000:
            break

    return multiply(part1), multiply(seen.values())


assert (11687500, 1) == pulse(
    """broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output""",
)

In [ ]:
p.answers = pulse(p.input_data)

# Day 21 - Step Counter

Part B broke me.  Not finished.

In [ ]:
p = Puzzle(day=21)

In [ ]:
def garden(input, moves=64, forever=False, output=None):
    data = dict(
        ((x, y), c)
        for y, line in enumerate(input.splitlines())
        for x, c in enumerate(line)
    )
    width, height = max(data.keys())
    max_gnomes = sum(1 for c in data.values() if c != "#")
    width = width + 1
    height = height + 1
    start = next(k for k, v in data.items() if v == "S")

    def step(gnomes, forever=False):
        key = tuple(gnomes)
        result = set()
        for g in gnomes:
            for n in neighbors4(g):
                x, y = n
                if forever:
                    x, y = x % width, y % height
                if data.get((x, y), "#") != "#":
                    result.add(n)
        return result

    gnomes = set([start])
    for move in range(moves):
        result = set()
        gnomes = step(gnomes, forever)
        if output is not None:
            print(f"{move+1},{len(gnomes)}", file=output)

    return len(gnomes)


assert 16 == garden(p.examples[0].input_data, 6)
# garden(p.examples[0].input_data, 200, True, open("day21.csv", "w"))

# Day 22 - Sand Slabs

In [ ]:
p = Puzzle(day=22)

In [ ]:
def overlap(x0, x1, x2, x3):
    minx = max(x0, x2)
    maxx = min(x1, x3)
    return minx <= maxx


def fall(slab, to_z):
    assert to_z <= slab[-1][-1]
    dz = slab[-1][-1] - slab[-1][0]
    return (slab[0], slab[1], (to_z, to_z + dz))


def countfalls(slab, beneath, above):
    fell = set()
    q = deque([slab])
    while q:
        slab = q.popleft()
        if slab in fell:
            continue
        for s in above[slab]:
            b = beneath[s] - fell
            assert slab in b
            if len(b) != 1:
                continue
            q.append(s)
        fell.add(slab)
    return len(fell) - 1


def slabs(input):
    data = [[vector(t) for t in line.split("~")] for line in input.splitlines()]
    slabs = [tuple(zip(p, q)) for p, q in data]
    falling = sorted(slabs, key=lambda x: x[-1][0])
    stable = []
    beneath = defaultdict(set)
    above = defaultdict(set)
    while falling:
        slab, falling = falling[0], falling[1:]
        under = [
            s for s in stable if overlap(*s[0], *slab[0]) and overlap(*s[1], *slab[1])
        ]
        minz = 1 + max([u[-1][-1] for u in under], default=0)
        if minz < slab[-1][-1]:
            slab = fall(slab, minz)
        for s in under:
            if s[-1][-1] == minz - 1:
                beneath[slab].add(s)
                above[s].add(slab)
        stable.append(slab)
    splode = []
    for slab in stable:
        a, b = above[slab], beneath[slab]
        if all(len(beneath[s]) > 1 for s in a):
            splode.append(slab)

    # Chain reactions
    fell = sum(countfalls(s, beneath, above) for s in stable)

    return len(splode), fell


assert (5, 7) == slabs(p.examples[0].input_data)

In [ ]:
p.answers = slabs(p.input_data)

# Day 23 - A Long Walk

Part 1 my (brute force) solution

In [ ]:
p = Puzzle(day=23)

In [ ]:
UP, DOWN, LEFT, RIGHT = (0, -1), (0, 1), (-1, 0), (1, 0)
STEPS = {
    ">": [RIGHT],
    "<": [LEFT],
    "v": [DOWN],
    "^": [UP],
    ".": [UP, DOWN, LEFT, RIGHT],
}


def longwalk(data, dry=False):
    maze = data.splitlines(0)
    assert maze[0].count(".") == 1 and maze[-1].count(".") == 1
    width, height = len(maze[0]), len(maze)
    start = (maze[0].index("."), 0)
    goal = (maze[-1].index("."), height - 1)
    best = 0
    work = deque()
    work.append([start])
    while work:
        path = work.popleft()
        pos = path[-1]
        if pos == goal:
            best = max(best, len(path) - 1)
            continue
        slope = maze[Y(pos)][X(pos)] if not dry else "."
        for dx, dy in STEPS[slope]:
            x, y = X(pos) + dx, Y(pos) + dy
            if x < 0 or x >= width:
                continue
            if y < 0 or y >= height:
                continue
            if maze[y][x] == "#":
                continue
            n = x, y
            if n in path:
                continue
            work.append(path + [(x, y)])
    return best


assert 94 == longwalk(p.examples[0].input_data)
assert 154 == longwalk(p.examples[0].input_data, True)

In [ ]:
longwalk(p.input_data)

## Part 2 needed help

In [ ]:
UP, DOWN, LEFT, RIGHT = (0, -1), (0, 1), (-1, 0), (1, 0)
STEPS = {
    ">": [RIGHT],
    "<": [LEFT],
    "v": [DOWN],
    "^": [UP],
    ".": [UP, DOWN, LEFT, RIGHT],
}

# Build subgraph with distances between each interesting node
# Core logic from https://github.com/mebeim/aoc/blob/master/2023/solutions/day23.py


def longwalk(data, dry=False):
    worst = defaultdict(int)
    maze = data.splitlines(0)
    assert maze[0].count(".") == 1 and maze[-1].count(".") == 1
    width, height = len(maze[0]), len(maze)
    start = (maze[0].index("."), 0)
    goal = (maze[-1].index("."), height - 1)

    def neighbors(p):
        x, y = p
        slope = maze[y][x]
        for dx, dy in STEPS[slope if not dry else "."]:
            xx, yy = x + dx, y + dy
            if 0 <= xx < width and 0 <= yy < height and maze[yy][xx] != "#":
                yield xx, yy

    def num_neighbors(p):
        return sum(1 for n in neighbors(p))

    def is_node(p):
        return p in (start, goal) or num_neighbors(p) > 2

    def adjacent_nodes(p):
        q = deque([(p, 0)])
        seen = set()
        while q:
            p, dist = q.popleft()
            seen.add(p)
            for n in neighbors(p):
                if n in seen:
                    continue
                if is_node(n):
                    yield (n, dist + 1)
                    continue
                q.append((n, dist + 1))

    def make_graph():
        graph = defaultdict(list)
        q = deque([start])
        seen = set()
        while q:
            pos = q.popleft()
            if pos in seen:
                continue
            seen.add(pos)
            for n, dist in adjacent_nodes(pos):
                graph[pos].append((n, dist))
                q.append(n)
        return graph

    def longest_path(graph, p, q, distance=0, seen=set()):
        if p == q:
            return distance
        best = 0
        seen.add(p)
        for n, dist in graph[p]:
            if n in seen:
                continue
            best = max(best, longest_path(graph, n, q, distance + dist, seen))
        seen.remove(p)
        return best

    graph = make_graph()
    return longest_path(graph, start, goal)


assert 94 == longwalk(p.examples[0].input_data)
assert 154 == longwalk(p.examples[0].input_data, True)

In [ ]:
p.answers = longwalk(p.input_data), longwalk(p.input_data, True)

# Day 24 - Never Tell Me The Odds

Very hard

In [ ]:
p = Puzzle(day=24)

In [ ]:
# Based on https://topaz.github.io/paste/#XQAAAQArBgAAAAAAAAAzHIoib6p4r/McpYgEEgWhHoa5LSRMkVi92ASWXgRJn/53YkZBUIS0YcdnfLPHqXGP79VwC0+MEZYxBj9+BVCmgt8T4zRMz8VlYFsdirsLgctk12iAq7FFAdeT7S6vWy3w7hb0v9dj8KQGFe0lrd7Caw7fc2qX6Ujxq0hSR8KZMLiOVMuTZjiw4SxtThSSZk5fAK4KXsQ3g9EaQ4K4oGTP/gOlxNoyS4gLbXvWyg733zZiUXc/ZvxkfLLEAl4eQFhi2PClu7B3xxmzv3oFziYm6Wp9dP61Z3b4VgYtEXgNE5/CebxNE6vDwFFP4E8x8OjvB70nrhVL/zhPBttkMUpI8rpV7+fl1BjuM1Zcj1HBeylgclycnKobfxGO6t/4ucvez7aPvQP5DxAU+HycjKW56BknLkHvSH3Sd5vzcX2WdLEN60Q62l3xs6vQHjwqEtagteOXCTpq+yjY5wqv61CwEZie3iRvFAuq3iBO3+mqdffk9GAmlPHt/DgHWRQ2X7Of6rOGdo6zDo1hHaTg/ejQpoLcf5Rzrszb0DJKte8sxTtPYhxDSghN/v8RhoQXqMj4ORPMsmTjWK0+zVa6LlvLD6Fn4T9iVNLElckjyyC7FwZQFQywT4pn3AvGwHYc7IjVsO6LDQPrBRNBJsce7TxuPRJSxNfPuIGLdjkYuOKQy0KQMoe4ZrdNyakWc+VaXsMP3CSACaQo0f5jYQb4j6X/XtF3r0exfyAqvXA7tvoE1S5/6ZnhK2HOxTXToirRdEP8vsZ1v7XItbm9wdNg0igF/wsR5L+rP2qSlxiyYlfSy0705b+5YnfMB10H+ozKHDh7zgvwtxERk8f5d/4KhPyHd32+bc+Lmrvpd7qXZf5yd/g=


def intersect(p, r, q, s, lo, hi):
    x1, y1 = p
    xv1, yv1 = r
    x2, y2 = q
    xv2, yv2 = s
    if yv1 * xv2 == yv2 * xv1:
        return False
    t1 = (yv2 * (x1 - x2) - xv2 * (y1 - y2)) / (yv1 * xv2 - xv1 * yv2)
    t2 = (yv1 * (x2 - x1) - xv1 * (y2 - y1)) / (yv2 * xv1 - xv2 * yv1)
    if t1 < 0 or t2 < 0:
        return False
    ix = x1 + t1 * xv1
    iy = y1 + t1 * yv1
    return lo <= ix <= hi and lo <= iy <= hi


def hail(input, lo, hi):
    data = array(input)
    data = [(tuple(r[:2]), tuple(r[4:6])) for r in data]
    part1 = 0
    for h1, h2 in combinations(data, 2):
        part1 += intersect(*h1, *h2, lo, hi)
    return part1


assert 2 == hail(p.examples[0].input_data, 7, 27)

In [ ]:
p.answer_a = hail(p.input_data, 200_000_000_000_000, 400_000_000_000_000)

In [ ]:
# Via https://pastebin.com/s6nvy0jA

import re
import numpy as np


def advent_day24_part2(input):
    field = [x for x in input.splitlines()]
    nrows, ncols = len(field), len(field[0])

    stones = []
    for line in field:
        m = re.split(r"[, @]+", line)
        if m:
            vals = [int(x) for x in m]
            p = tuple(vals[:3])
            v = tuple(vals[3:])
            stones.append((p, v))

    # x = [sx sy sz wx wy wz t1 t2 ... tn]
    # x_n+1 = x_n - J(x_n)^-1 f(x_n)
    # f(x) = sx + wx * t1 - (px1 + vx1 * t1)
    #      = (sx - px1) + (wx - vx1) * t1
    # ... and again for y and z
    # ... and again for all n
    # J_x = [ 1  0  0 t1  0  0  wx-vx1  0  0 ...]
    # J_y = [ 0  1  0  0 t1  0  wy-vy1  0  0 ...]
    # J_z = [ 0  0  1  0  0 t1  wz-vz1  0  0 ...]

    ps = np.array(tuple(row[0] for row in stones), dtype=np.float64)
    vs = np.array(tuple(row[1] for row in stones), dtype=np.float64)
    x = np.array([0, 0, 0, 1, 1, 1] + [0] * nrows, dtype=np.float64)

    def jacobian(x, vs):
        data = np.zeros((nrows * 3, 6 + nrows))
        for i in range(nrows):
            for j in range(3):
                r = 3 * i + j
                data[r, j] = 1
                data[r, 3 + j] = x[6 + i]
                data[r, 6 + i] = x[3 + j] - vs[i][j]
        return data

    def func(x, ps, vs):
        data = np.zeros(nrows * 3)
        for i in range(nrows):
            for j in range(3):
                r = 3 * i + j
                t = x[6 + i]
                sp = x[j] + x[3 + j] * t
                hp = ps[i][j] + vs[i][j] * t
                data[r] = sp - hp
        return data

    sum0 = 0
    for i in range(100):
        j = jacobian(x, vs)
        f = func(x, ps, vs)
        dx, resid, rank, sing = np.linalg.lstsq(j, f, rcond=None)
        x -= dx
        sum1 = np.sum(x[:3])
        if abs(sum1 - sum0) < 0.1:
            break
        sum0 = sum1

    return int(sum0)

In [ ]:
p.answer_b = advent_day24_part2(p.input_data)

# Day 25 - Snowverload

In [ ]:
p = Puzzle(day=25)

In [ ]:
def cutgraph(input):
    # Construct graph
    graph = defaultdict(list)
    edges = set()
    for line in input.splitlines():
        node1, connected = line.split(": ")
        for node2 in connected.split():
            graph[node1].append(node2)
            graph[node2].append(node1)
            edges.add(tuple(sorted((node1, node2))))

    # Calculate edge betweenness centrality
    edge_betweenness = defaultdict(int)
    for node in graph:
        visited = set([node])
        queue = deque([(node, [])])
        while queue:
            for _ in range(len(queue)):
                curr, path = queue.popleft()
                for edge in path:
                    edge_betweenness[edge] += 1
                for nxt in graph[curr]:
                    if nxt not in visited:
                        visited.add(nxt)
                        edge = tuple(sorted((nxt, curr)))
                        queue.append((nxt, path + [edge]))

    # Remove the three most frequently travelled edges from graph
    most_crucial_edges = sorted(edge_betweenness, key=edge_betweenness.get)[-3:]
    for node1, node2 in most_crucial_edges:
        graph[node1].remove(node2)
        graph[node2].remove(node1)

    # Traverse one of the resulting smaller graphs to find out its size
    visited = set([node])
    queue = [node]
    while queue:
        curr = queue.pop()
        for nxt in graph[curr]:
            if nxt not in visited:
                visited.add(nxt)
                queue.append(nxt)

    # Infer graph sizes and return their product
    size1 = len(visited)
    size2 = len(graph) - size1
    return size1 * size2


assert 54 == cutgraph(
    """jqt: rhn xhk nvd
rsh: frs pzl lsr
xhk: hfx
cmg: qnr nvd lhk bvb
rhn: xhk bvb hfx
bvb: xhk hfx
pzl: lsr hfx nvd
qnr: nvd
ntq: jqt hfx bvb xhk
nvd: lhk
lsr: lhk
rzs: qnr cmg lsr rsh
frs: qnr lhk lsr"""
)

In [ ]:
p.answer_a = cutgraph(p.input_data)

In [ ]:
def cutgraph(input):
    # Construct graph
    graph = defaultdict(list)
    for line in input.splitlines():
        node1, connected = line.split(": ")
        for node2 in connected.split():
            graph[node1].append(node2)
            graph[node2].append(node1)

    # Calculate edge betweenness centrality
    edge_betweenness = Counter()
    for node in graph:
        visited = set([node])
        queue = deque([(node, [])])
        while queue:
            for _ in range(len(queue)):
                curr, path = queue.popleft()
                edge_betweenness.update(path)
                for nxt in graph[curr]:
                    if nxt not in visited:
                        visited.add(nxt)
                        edge = tuple(sorted((nxt, curr)))
                        queue.append((nxt, path + [edge]))

    # Remove the three most frequently travelled edges from graph
    most_crucial_edges = edge_betweenness.most_common(3)
    print(most_crucial_edges)
    for (node1, node2), count in most_crucial_edges:
        graph[node1].remove(node2)
        graph[node2].remove(node1)

    # Traverse one of the resulting smaller graphs to find out its size
    node = first(graph.keys())
    visited = set([node])
    queue = [node]
    while queue:
        curr = queue.pop()
        for nxt in graph[curr]:
            if nxt not in visited:
                visited.add(nxt)
                queue.append(nxt)

    # Infer graph sizes and return their product
    size1 = len(visited)
    size2 = len(graph) - size1
    return size1 * size2


assert 54 == cutgraph(
    """jqt: rhn xhk nvd
rsh: frs pzl lsr
xhk: hfx
cmg: qnr nvd lhk bvb
rhn: xhk bvb hfx
bvb: xhk hfx
pzl: lsr hfx nvd
qnr: nvd
ntq: jqt hfx bvb xhk
nvd: lhk
lsr: lhk
rzs: qnr cmg lsr rsh
frs: qnr lhk lsr"""
)

In [ ]:
cutgraph(p.input_data)